In [1]:
import csv
import numpy as np
from datetime import datetime


In [2]:
data = []
with open('gv-data/trajectories.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        row['time'] = float(row['time'])  # Convert 'time' to float
        row['cell_index'] = int(row['cell_index'])  # Convert 'cell_index' to int
        data.append(dict(row))


cell_index_to_info = []
with open('gv-data/cell_towers.csv', mode='r') as file:
    csv_reader = csv.DictReader(file)
    
    for row in csv_reader:
        cell_index = row['cell_index']
        cell_index_to_info.append({
            'mcc': row['mcc'],
            'mnc': row['mnc'],
            'area_code': row['area_code'],
            'cell_id': row['cell_id']
        })


In [3]:
times = np.array([row['time'] for row in data])
uuids = np.array([row['device_uuid'] for row in data])
towers = np.array([row['cell_index'] for row in data])

uuid_time_diff = []
uuid_time_stamps = []
uuid_towers = []

for uuid in np.unique(uuids):
    indices = np.where(uuids == uuid)[0]
    uuid_time_diff.append(np.diff(times[indices]))
    uuid_time_stamps.append(times[indices])
    uuid_towers.append(towers[indices])


In [4]:
# Get bin indices of an array of epoch times, where a day is split into bin_count bins
def bin_indices_from_epoch(epoch, bin_count):
    if isinstance(epoch, (int, float)):
        epoch = [epoch]
        single_value = True
    else:
        single_value = False
    
    bin_size = 60 * 60 * 24 / bin_count
    date_times = [datetime.fromtimestamp(epoch_i) for epoch_i in epoch]
    seconds_since_midnight = np.array([dt.hour * 3600 + dt.minute * 60 + dt.second for dt in date_times])
    array = (seconds_since_midnight / bin_size).astype(int)
    
    return array[0] if single_value else array


# Generate timestamps for a synthetic trajectory. Randomly samples from bins.
def generate_timestamps(start_time, end_time, time_diff_bins, bin_count):
    trajectory = [start_time]
    while trajectory[-1] < end_time:
        bin_i = bin_indices_from_epoch(trajectory[-1], bin_count)
        trajectory.append(trajectory[-1] + np.random.choice(time_diff_bins[bin_i]))
    
    trajectory = np.array(trajectory)[:-1]
    return trajectory


# Get randomized start and stop times based on existing starts and stops
def get_start_stop(uuid_time_stamps, generate_num):
    mins = np.array([min(tr) for tr in uuid_time_stamps])
    maxs = np.array([max(tr) for tr in uuid_time_stamps])

    start_mean, start_std = np.mean(mins), np.std(mins)
    end_mean, end_std = np.mean(maxs), np.std(maxs)
    starts = np.random.normal(start_mean, start_std, generate_num)
    ends = np.random.normal(end_mean, end_std, generate_num)
    
    # Ensure stop times are always after start times
    start_stop = [(min(s, e), max(s, e)) for s, e in zip(starts, ends)]
    
    return start_stop


# Generate tower trajectory given probabilities, times, and an initial tower
def generate_towers(start_tower, time_trajectory, m_probabilities, bin_count):
    trajectory = [start_tower]
    bin_trajectory = bin_indices_from_epoch(time_trajectory, bin_count)
    
    for bin_i in bin_trajectory[1:]:
        current_tower = trajectory[-1]
        
        # use adjacent bins if this time bin is empty
        bin_radius = 0
        max_radius = bin_count // 2  # Prevent infinite loop
        
        probabilities = np.zeros(m_probabilities.shape[-1])
        while bin_radius <= max_radius:
            lower_bin = (bin_i - bin_radius) % bin_count
            upper_bin = (bin_i + bin_radius) % bin_count
            
            if bin_radius == 0:
                probabilities = m_probabilities[current_tower][lower_bin]
            else:
                lower_prob = m_probabilities[current_tower][lower_bin]
                upper_prob = m_probabilities[current_tower][upper_bin]
                if np.sum(lower_prob) == 0:
                    probabilities = upper_prob
                elif np.sum(upper_prob) == 0:
                    probabilities = lower_prob
                else:
                    probabilities = np.mean([lower_prob, upper_prob], axis=0)

            if np.sum(probabilities) > 0:
                break
            
            bin_radius += 1
        
        next_tower = np.random.choice(len(probabilities), p=probabilities)
        trajectory.append(next_tower)
    
    return trajectory


# Generate tower trajectory given probabilities, times, and an initial tower
def generate_towers_with_reversal(start_tower, time_trajectory, m_probabilities, m_probabilities_reversed, bin_count, expected_reversals_per_day=5):
    trajectory = [start_tower]
    bin_trajectory = bin_indices_from_epoch(time_trajectory, bin_count)
    
    days = (time_trajectory[-1] - time_trajectory[0]) / (24 * 60 * 60)
    reversal_probability = expected_reversals_per_day * days / len(time_trajectory)
    current_reversed = False
    reversal_probabilities = m_probabilities
    
    for bin_i in bin_trajectory[1:]:
        if np.random.rand() < reversal_probability:
            if not current_reversed:
                current_reversed = True
                reversal_probabilities = m_probabilities_reversed
            else:
                current_reversed = False
                reversal_probabilities = m_probabilities
        
        current_tower = trajectory[-1]
        
        # use adjacent bins if this time bin is empty
        bin_radius = 0
        max_radius = bin_count // 2  # Prevent infinite loop
        
        probabilities = np.zeros(reversal_probabilities.shape[-1])
        while bin_radius <= max_radius:
            lower_bin = (bin_i - bin_radius) % bin_count
            upper_bin = (bin_i + bin_radius) % bin_count
            
            if bin_radius == 0:
                probabilities = reversal_probabilities[current_tower][lower_bin]
            else:
                lower_prob = reversal_probabilities[current_tower][lower_bin]
                upper_prob = reversal_probabilities[current_tower][upper_bin]
                if np.sum(lower_prob) == 0:
                    probabilities = upper_prob
                elif np.sum(upper_prob) == 0:
                    probabilities = lower_prob
                else:
                    probabilities = np.mean([lower_prob, upper_prob], axis=0)

            if np.sum(probabilities) > 0:
                break
            
            bin_radius += 1
        
        next_tower = np.random.choice(len(probabilities), p=probabilities)
        trajectory.append(next_tower)
    
    return trajectory


In [5]:
# Find time diffs for random choice as a function of time of day

num_bins = 48
bins = []
concatenated_diffs = np.concatenate(uuid_time_diff)
concatenated_time_stamps = np.concatenate([time_stamps[:-1] for time_stamps in uuid_time_stamps])

# Filter to sub 12 hour time diffs
concatenated_diffs_filtered = concatenated_diffs[concatenated_diffs < 12 * 60 * 60]
concatenated_time_stamps_filtered = concatenated_time_stamps[concatenated_diffs < 12 * 60 * 60]

bin_indices = bin_indices_from_epoch(concatenated_time_stamps_filtered, num_bins)
for i in range(num_bins):
    bins.append(concatenated_diffs_filtered[bin_indices == i])
    if bins[i].size == 0:
        raise RuntimeError("No values found for bin {}".format(i))

In [7]:
# Define markov probabilities over cell tower and time of day

num_towers = np.max(towers)

# Define probabilities of moving from one tower to another at a given time of day
markov_probabilities = np.zeros((num_towers, num_bins, num_towers))
markov_probabilities_reversed = np.zeros((num_towers, num_bins, num_towers))

for i in range(len(uuid_towers)):
    tower_tr = uuid_towers[i]
    bin_indices = bin_indices_from_epoch(uuid_time_stamps[i], num_bins)
    
    for j in range(len(tower_tr) - 1):
        from_tower = tower_tr[j] - 1
        to_tower = tower_tr[j + 1] - 1
        time_bin = bin_indices[j]
        
        markov_probabilities[from_tower, time_bin, to_tower] += 1
        markov_probabilities_reversed[to_tower, time_bin, from_tower] += 1

with np.errstate(divide='ignore', invalid='ignore'):
    markov_probabilities = np.divide(markov_probabilities, np.sum(markov_probabilities, axis=2, keepdims=True))
    markov_probabilities = np.nan_to_num(markov_probabilities)  
    markov_probabilities_reversed = np.divide(markov_probabilities_reversed, np.sum(markov_probabilities_reversed, axis=2, keepdims=True))
    markov_probabilities_reversed = np.nan_to_num(markov_probabilities_reversed)  
    
for i in range(num_towers):
    if np.sum(markov_probabilities[i]) == 0.0:
        print("no exits from tower ", i)
    if np.sum(markov_probabilities_reversed[i]) == 0.0:
        print("no entries to tower ", i)

# TODO: Implement sparse matrix. Only 8% of values are nonzero with 48 bins.


In [12]:
# Generate one sequence of data and export to a csv
# Note: Result is about 75kB per person-day or 5kB after tar.xz compression or 7kB after zip compression

start = datetime(2024, 7, 1).timestamp()
stop = datetime(2024, 7, 16).timestamp()
initial_tower = np.random.choice([tower_tr[0] for tower_tr in uuid_towers])

times = generate_timestamps(start, stop, bins, num_bins)

towers = generate_towers_with_reversal(initial_tower, times, markov_probabilities, markov_probabilities_reversed, num_bins)

formatted_trajectory = [{
    'uuid': '4ea0f558-51cb-45e3-94f9-9a0145ab5930',  # Synthetic-02 uuid
    'time': round(times[i], 3),
    'mcc': cell_index_to_info[towers[i - 1]]['mcc'],
    'mnc': cell_index_to_info[towers[i - 1]]['mnc'],
    'area_code': cell_index_to_info[towers[i - 1]]['area_code'],
    'cell_id': cell_index_to_info[towers[i - 1]]['cell_id']
} for i in range(len(times))]

csv_file_name = 'gv-data/synthetic/synthetic_reversal.csv'

# Define the field names for the CSV
fieldnames = ['uuid', 'time', 'mcc', 'mnc', 'area_code', 'cell_id']

# Write to the CSV file
with open(csv_file_name, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # Write the header
    writer.writeheader()
    
    # Write the data rows
    for row in formatted_trajectory:
        writer.writerow(row)
